In [ ]:
!pip install -q transformers datasets librosa evaluate jiwer gradio bitsandbytes accelerate peft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 MB 50.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.8/319.8 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.7/94.7 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.4/447.4 kB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
from huggingface_hub import list_datasets
from datasets import(load_dataset, DatasetDict, load_from_disk,
                     Audio, IterableDatasetDict, Features, Value,
                     Sequence, Dataset)
import IPython.display as ipd
from functools import partial

In [ ]:
voice_data = load_dataset("clt013/malay-speech-3k-rows-dataset_v2", token=True)
voice_data

README.md:   0%|          | 0.00/460 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/216M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/59.7M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2544 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/604 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['sentence', 'audio'],
        num_rows: 2544
    })
    test: Dataset({
        features: ['sentence', 'audio'],
        num_rows: 604
    })
})

In [ ]:
from transformers import WhisperProcessor
model_name = "openai/whisper-large-v3"
language = "malay"
task = "transcribe"
processor = WhisperProcessor.from_pretrained(model_name, language=language, task=task)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

preprocessor_config.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.07k [00:00<?, ?B/s]

In [ ]:
def prepare_dataset(batch):
    # load audio
    audio = batch["audio"]

    # compute log-Mel input features from input audio array
    batch["input_features"] = processor.feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # compute input length of audio sample in seconds
    batch["input_length"] = len(audio["array"]) / audio["sampling_rate"]

    # encode target text to label ids
    batch["labels"] = processor.tokenizer(batch["sentence"]).input_ids
    return batch



def is_audio_in_length_range(length):
    return 1.0 < length <= 30.0

def prepare_dataset_done():
  dt = voice_data.map(prepare_dataset,remove_columns=voice_data["train"].column_names).with_format("torch")
  dt["train"] = dt["train"].filter(
    is_audio_in_length_range,
    input_columns=["input_length"],
  )
  dt["test"] = dt["test"].filter(
    is_audio_in_length_range,
    input_columns=["input_length"],
  )
  return dt

In [ ]:
voice_data_new = prepare_dataset_done()
voice_data_new = voice_data_new.shuffle(seed=42)
voice_data_new

Map:   0%|          | 0/2544 [00:00<?, ? examples/s]

Map:   0%|          | 0/604 [00:00<?, ? examples/s]

Filter:   0%|          | 0/2544 [00:00<?, ? examples/s]

Filter:   0%|          | 0/604 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_features', 'input_length', 'labels'],
        num_rows: 2139
    })
    test: Dataset({
        features: ['input_features', 'input_length', 'labels'],
        num_rows: 513
    })
})

In [ ]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained(model_name,  load_in_8bit=True, device_map="auto")

config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.90k [00:00<?, ?B/s]

In [ ]:
model.generation_config.language = "malay"
model.generation_config.task = "transcribe"
model.generation_config.forced_decoder_ids = None
model.generation_config.suppress_tokens = []
model.config.use_cache = False

In [ ]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")
        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

In [ ]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

In [ ]:
import evaluate

metric = evaluate.load("wer")

In [ ]:
import peft
from peft import prepare_model_for_kbit_training

model = prepare_model_for_kbit_training(model)

In [ ]:
def make_inputs_require_grad(module, input, output):
    output.requires_grad_(True)

model.model.encoder.conv1.register_forward_hook(make_inputs_require_grad)

In [ ]:
from peft import LoraConfig, PeftModel, LoraModel, LoraConfig, get_peft_model

config = LoraConfig(r=32, lora_alpha=64, target_modules=["q_proj", "v_proj"], lora_dropout=0.05, bias="none")

model = get_peft_model(model, config)
model.print_trainable_parameters()

trainable params: 15,728,640 || all params: 1,559,219,200 || trainable%: 1.0088


In [ ]:
!rm -rf "./whisper-small-ft-malay-peft-v1"

In [ ]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-large-v3-ft-malay-peft-epoch-20",  # change to a repo name of your choice
    per_device_train_batch_size=8,
    gradient_accumulation_steps=1,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-05,
    warmup_steps=100,
    num_train_epochs=20,
    fp16=True,
    eval_strategy="steps",
    per_device_eval_batch_size=8,
    generation_max_length=128,
    # save_steps=1000,
    # eval_steps=1000,
    logging_steps=100,
    report_to=["tensorboard"],
    remove_unused_columns=False,
    label_names=["labels"]
)

In [ ]:
from transformers import Seq2SeqTrainer, TrainerCallback, TrainingArguments, TrainerState, TrainerControl
from transformers.trainer_utils import PREFIX_CHECKPOINT_DIR
import os

# This callback helps to save only the adapter weights and remove the base model weights.
class SavePeftModelCallback(TrainerCallback):
    def on_save(
        self,
        args: TrainingArguments,
        state: TrainerState,
        control: TrainerControl,
        **kwargs,
    ):
        checkpoint_folder = os.path.join(args.output_dir, f"{PREFIX_CHECKPOINT_DIR}-{state.global_step}")

        peft_model_path = os.path.join(checkpoint_folder, "adapter_model")
        kwargs["model"].save_pretrained(peft_model_path)

        pytorch_model_path = os.path.join(checkpoint_folder, "pytorch_model.bin")
        if os.path.exists(pytorch_model_path):
            os.remove(pytorch_model_path)
        return control

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=voice_data_new["train"],
    eval_dataset=voice_data_new["test"],
    data_collator=data_collator,
    tokenizer=processor.feature_extractor,
    callbacks=[SavePeftModelCallback],
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:316: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cp

Step,Training Loss,Validation Loss
100,2.625700,2.425809
200,2.339400,1.762923
300,1.525600,1.198272
400,1.064800,0.911123
500,0.745800,0.575441
600,0.528700,0.531564
700,1.022100,1.630769
800,0.904500,0.465480
900,0.376600,0.418588
1000,0.376700,0.408191


/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:316: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cp

TrainOutput(global_step=5360, training_loss=0.40170366870823193, metrics={'train_runtime': 46768.9999, 'train_samples_per_second': 0.915, 'train_steps_per_second': 0.115, 'total_flos': 1.468952728436736e+20, 'train_loss': 0.40170366870823193, 'epoch': 20.0})

In [ ]:
kwargs = {
    "dataset_tags": "clt013/malay-speech-3k-rows-dataset_v2",
    "dataset": "Malay Speech 3k",  # a 'pretty' name for the training dataset
    "language": "ms",
    "model_name": "Whisper Large v3 FT Malay - CLT013",  # a 'pretty' name for your model
    "finetuned_from": model_name,
    "tasks": "automatic-speech-recognition",
}
trainer.push_to_hub(**kwargs)
repo_id = "clt013/whisper-large-v3-ft-malay-peft-epoch-20"
model.push_to_hub(repo_id)

adapter_model.safetensors:   0%|          | 0.00/63.0M [00:00<?, ?B/s]

events.out.tfevents.1729421160.cb4d8a5b1987.735.0:   0%|          | 0.00/32.1k [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

training_args.bin:   0%|          | 0.00/5.37k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.28k [00:00<?, ?B/s]

No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/clt013/whisper-large-v3-ft-malay-peft-epoch-20/commit/a6c60997c65db1c5682c2dbd743feb7c63da20fc', commit_message='Upload model', commit_description='', oid='a6c60997c65db1c5682c2dbd743feb7c63da20fc', pr_url=None, repo_url=RepoUrl('https://huggingface.co/clt013/whisper-large-v3-ft-malay-peft-epoch-20', endpoint='https://huggingface.co', repo_type='model', repo_id='clt013/whisper-large-v3-ft-malay-peft-epoch-20'), pr_revision=None, pr_num=None)

In [ ]:
from peft import PeftModel, PeftConfig
from transformers import WhisperForConditionalGeneration, Seq2SeqTrainer

peft_model_id = repo_id
peft_config = PeftConfig.from_pretrained(peft_model_id)
model_loaded = WhisperForConditionalGeneration.from_pretrained(
    peft_config.base_model_name_or_path, load_in_8bit=True, device_map="auto"
)
model_loaded = PeftModel.from_pretrained(model_loaded, peft_model_id)

adapter_config.json:   0%|          | 0.00/771 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


adapter_model.safetensors:   0%|          | 0.00/63.0M [00:00<?, ?B/s]

In [ ]:
from torch.utils.data import DataLoader
from tqdm import tqdm
import numpy as np
import gc

eval_dataloader = DataLoader(voice_data_new["test"], batch_size=8, collate_fn=data_collator)

model.eval()
for step, batch in enumerate(tqdm(eval_dataloader)):
    with torch.cuda.amp.autocast():
        with torch.no_grad():
            generated_tokens = (
                model.generate(
                    input_features=batch["input_features"].to("cuda"),
                    decoder_input_ids=batch["labels"][:, :4].to("cuda"),
                    max_new_tokens=255,
                )
                .cpu()
                .numpy()
            )
            labels = batch["labels"].cpu().numpy()
            labels = np.where(labels != -100, labels, processor.tokenizer.pad_token_id)
            decoded_preds = processor.tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
            decoded_labels = processor.tokenizer.batch_decode(labels, skip_special_tokens=True)
            metric.add_batch(
                predictions=decoded_preds,
                references=decoded_labels,
            )
            print(step)
            print(decoded_preds)
            print(decoded_labels)
    del generated_tokens, labels, batch
    gc.collect()
wer = 100 * metric.compute()
print(f"{wer=}")


  0%|          | 0/65 [00:00<?, ?it/s]<ipython-input-20-d2fde27b1a1a>:10: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:316: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


0
['ha', 'H HP laptop HP', 'tarik tarik', 'sebab Huawei pun centre sini kurang kan kalau macam Samsung', 'badminton sebetul-betul badminton', 'kegemaran aku', 'mengelakkan daripada masalah masalah ni berlaku lah macam ah kesihatan kecemaannya semua ni sebab dia sangat membahaya lah sebab', 'kemurungan yang boleh berlaku dalam family']
['ah', 'HP laptop HP tu', 'tarik tarik', 'sebab Huawei pun centre sini kurang kan kalau macam Samsung', 'badminton sudah tentu', 'kegemaran aku', 'mengelakkan daripada masalah-masalah ni berlakulah macam ah kesihatan kencing manis semua ni sebab dia sangat berbahaya la sebab', 'kemurungan boleh berlaku di dalam family']


  2%|▏         | 1/65 [00:13<14:13, 13.33s/it]

1
['kalau nak makan tu memilih so aku macam tak tahu sangat pasal makanan ni', 'tosai ni kaya dengan karbohidrat protein', 'oh', 'kan ada banyak macam ISIS itu kan banyak dunia ni', 'makanan', 'um um okey nanti kalau', 'yang untuk makan makanan makan besar tu kan', 'ha boleh lah awak tahu bagitahulah apa sebab ah adik saya terkena demam tersebut apa bila bermain bermadi hujan']
['kalau nak makan tu memilih so aku macam tak tahu sangat pasal makanan ni', 'tosai ni kaya dengan karbohidrat protein', 'oh', 'kan ada banyak macam search situ kan banyak benda ni', 'makanan eh', 'um um okey nanti kalau', 'yang untuk makan makanan makan besar tu kan', 'ah boleh awak tahu bagitahu lah apa sebab ah adik saya terkena demam tersebut apabila bermain bermandi hujan']


  3%|▎         | 2/65 [00:24<12:57, 12.33s/it]

...

63
['ha nurse nurse datang lepas tu bagi enjet eh', 'sekiranya tak dapat kawal', 'pemakanan jugalah tapi saya tidak tahu ah dari mana pemakanan itu ah boleh berlakunya Covid nineteen lah sebab', 'sedangkan berenang mengambil', 'ha kuih baulu aku suka sangat kuih baulu ni', 'taksa tu berbeza sikit', 'pergilah kat rumah kau kan nak ah nanti', 'aku mungkin kalau boleh nak cari yang apa ni']
['ha nurse nurse datang lepas tu bagi inject kan', 'kiranya tak dapat kawal', 'pemakanan jugalah tapi saya tidak tahu ah dari mana pemakanan itu boleh berlakunya Covid ninteen la sebab', 'sedangkan berenang mengambil', 'ha Kuih Bahulu aku suka sangat Kuih Bahulu ni', '[UNK] situ berbeza sikit', 'pergilah kat rumah kau kan nak ah nanti', 'aku ingat kalau boleh nak cari yang apa ni']


 98%|█████████▊| 64/65 [13:41<00:09,  9.25s/it]

64
['okey okey okey ah']
['okay okay okay ah']


100%|██████████| 65/65 [13:43<00:00, 12.68s/it]

wer=28.772635814889334
